# Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Load & Prepare Data

In [ ]:
# Simulated structure of your DataFrame
# df = pd.read_csv('your_data.csv')  # If loading from file

# Define features and targets
categorical_features = ['machine_type', 'glue_type', 'wood_type']  # replace with actual
numerical_features = ['temperature', 'pressure', 'moisture', 'speed']  # replace with actual

target_columns = ['accuracy', 'tensile_strength', 'waste_material']

# Ensure correct types
for cat in categorical_features:
    df[cat] = df[cat].astype('category')

X = df[categorical_features + numerical_features]
y = df[target_columns]

# Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Separate GBDT Models per Target (Multivariate Regression via multiple models)

In [ ]:
models = {}
predictions = {}
metrics = {}

for target in target_columns:
    train_data = lgb.Dataset(X_train, label=y_train[target], categorical_feature=categorical_features)
    test_data = lgb.Dataset(X_test, label=y_test[target], categorical_feature=categorical_features)

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'verbose': -1
    }

    model = lgb.train(params, train_data, valid_sets=[test_data], early_stopping_rounds=20, verbose_eval=False)
    models[target] = model

    # Predict and evaluate
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    predictions[target] = y_pred
    metrics[target] = {
        'RMSE': mean_squared_error(y_test[target], y_pred, squared=False),
        'R2': r2_score(y_test[target], y_pred)
    }

# Print Evaluation Metrics

In [ ]:
for target in target_columns:
    print(f"\n{target.upper()}")
    print("RMSE:", round(metrics[target]['RMSE'], 3))
    print("R2 Score:", round(metrics[target]['R2'], 3))

# Feature Importance (Optional)

In [ ]:
import matplotlib.pyplot as plt

for target in target_columns:
    lgb.plot_importance(models[target], max_num_features=10, title=f"Feature Importance for {target}")
    plt.show()